In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [2]:
%%time 
import warnings
warnings.filterwarnings('ignore')
import glob
import numpy as np 
import argparse 
import importlib
import os 
from os import path
import datetime

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import tensorflow as tf ; print("You have imported tensorflow version :", tf.__version__)
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

import keras 
print("Keras version : ",keras.__version__)
from tensorflow.keras.optimizers import Adam , Adagrad, Adadelta, RMSprop, SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar100 , cifar10
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import EfficientNetB0


You have imported tensorflow version : 2.4.1
Keras version :  2.3.1
CPU times: user 3.35 s, sys: 2.72 s, total: 6.07 s
Wall time: 2.53 s


Using TensorFlow backend.


### The Image data generators for train test and validation 

In [9]:
%%time 

train_directory = '/home/aastha/imagenet/data/official_data/train/'
batch_size = 64

datagen = ImageDataGenerator(rescale=1./255.,  validation_split=0.05 ,   shear_range=0.2, zoom_range=0.2,horizontal_flip=True)


train_it = datagen.flow_from_directory( train_directory  , target_size=(224, 224), color_mode="rgb", batch_size=batch_size, class_mode="categorical", shuffle=True, seed=42, subset="training")
val_it = datagen.flow_from_directory(train_directory, target_size=(224, 224), color_mode="rgb", batch_size=batch_size, class_mode="categorical", shuffle=True, seed=42, subset="validation")

Found 1198811 images belonging to 1000 classes.
Found 62594 images belonging to 1000 classes.
CPU times: user 1min 7s, sys: 31.6 s, total: 1min 38s
Wall time: 1min 3s


### The LR Scheduler 

In [4]:
%%time 
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

# did not use this in callbacks 
# anne = ReduceLROnPlateau(monitor='categorical_accuracy', factor=0.5, patience=1, verbose=1, min_lr=1e-6)

CPU times: user 30 µs, sys: 0 ns, total: 30 µs
Wall time: 35.3 µs


### End of epoch 

In [5]:
%%time 
class PrintLR(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs=None):
            model.save("b0_210203_model_continued1.h5") # the model is saved at the end of each epoch
            print('\nLearning rate for epoch {} is {}'.format(epoch + 1,model.optimizer.lr.numpy()))
            print("Loss :", logs['loss'])
            print("Accuracy :",logs['categorical_accuracy'])
            print()


CPU times: user 28 µs, sys: 0 ns, total: 28 µs
Wall time: 30.3 µs


### Checkpointing and tensorboard setup 

In [6]:
%%time 
checkpoint_dir = './training_checkpoints_210202_b0'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

CPU times: user 32 µs, sys: 0 ns, total: 32 µs
Wall time: 34.8 µs


In [7]:
%%time 
callbacks = [ 
    PrintLR(),
    lr_callback,
    tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,save_best_only=True , verbose=1)
]


CPU times: user 10.7 ms, sys: 42.8 ms, total: 53.5 ms
Wall time: 184 ms


In [12]:
%%time 
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

with strategy.scope():
    if path.exists("b0_210203_model_continued.h5"):
        model = tf.keras.models.load_model("b0_210203_model_continued.h5")
        print("Loading old model"); print()
    else:
        model = EfficientNetB0(weights=None)    
        
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True , name='categorical_crossentropy'), # change this to sparse=cce and metric to sparceCategoricalAccuracy 
        metrics=[tf.keras.metrics.CategoricalAccuracy()] )
    
    train_hist = model.fit_generator(
        train_it,  epochs=1, verbose=1, callbacks= callbacks,steps_per_epoch=train_it.samples // batch_size,
        validation_data= val_it,  validation_freq=1,validation_steps=val_it.samples // batch_size, shuffle=True)


    model.save("b0_210203_model_continued1.h5")

    print(); print(); print("Loss vs epochs data - \n", train_hist.history["loss"] )
    print()
    print("Accuracy vs epochs data - \n", train_hist.history["categorical_accuracy"] )
    print()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices: 1
Loading old model

2/2 [==============================] - 23s 9s/step - loss: 0.9766 - categorical_accuracy: 0.7344 - val_loss: 1.5144 - val_categorical_accuracy: 0.6641

Learning rate for epoch 1 is 9.999999747378752e-05
Loss : 0.9375073313713074
Accuracy : 0.75


Epoch 00001: val_loss improved from inf to 1.51444, saving model to ./training_checkpoints_210202_b0/ckpt_1
INFO:tensorflow:Assets written to: ./training_checkpoints_210202_b0/ckpt_1/assets


Loss vs epochs data - 
 [0.9375073313713074]

Accuracy vs epochs data - 
 [0.75]

CPU times: user 44.4 s, sys: 343 ms, total: 44.8 s
Wall time: 44.4 s
